<a href="https://colab.research.google.com/github/Geekomaniac1009/PCR_Trading_Strategy/blob/main/KritiQuant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

In [ ]:
pip install ta

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [ ]:
def pnl_ratio(signals, share_ratio, num_shares):

    initial_cap = 100000


    portfolio = pd.DataFrame()
    portfolio['holdings'][0] = 0
    portfolio['shares'][0] = 0
    portfolio['positions'] = signals['position']
    for idx in range(len(signals)):
        #buy posn
        if signals['position'][idx] == 1:
            shares = initial_cap*share_ratio // signals['Close'][idx]
            initial_cap = initial_cap - shares*signals['Close'][idx]
            if idx != 0:
                portfolio['shares'][idx] = portfolio['shares'][idx-1] + shares
            else:
                portfolio['shares'][idx] = shares

        #sell posn
        elif signals['position'][idx] == -1 and num_shares <= portfolio['shares'][idx-1] and idx > 0:
            initial_cap = initial_cap + num_shares*signals['Close'][idx]
            portfolio['shares'][idx] = portfolio['shares'][idx-1] - num_shares

        #hold posn
        elif signals['position'][idx] == 0:
            portfolio['shares'][idx] = portfolio['shares'][idx-1]

        portfolio['cash'][idx] = initial_cap
        portfolio['holdings'][idx] = portfolio['shares'][idx]*signals['Close'][idx]
        portfolio['total'][idx] = portfolio['cash'][idx] + portfolio['holdings'][idx]
    portfolio['returns'] = portfolio['total'].pct_change()
    return portfolio

In [ ]:
def sharpe_ratio(portfolio, risk_free_rate):
    avg_return = np.mean(portfolio['returns'])
    std_dev = np.std(portfolio['returns'])
    sharpe = (avg_return - risk_free_rate) / std_dev
    return sharpe

In [ ]:
def beta(portfolio, benchmark):
    covariance = np.cov(portfolio['returns'], benchmark['returns'])
    beta = covariance[0,1] / np.var(benchmark['returns'])
    return beta

In [ ]:
def alpha(portfolio, benchmark, risk_free_rate):
    avg_return = np.mean(portfolio['returns'])
    benchmark_avg_return = np.mean(benchmark['returns'])
    beta = beta(portfolio, benchmark)
    alpha = avg_return - risk_free_rate - beta * (benchmark_avg_return - risk_free_rate)
    return alpha

In [ ]:
def sortino_ratio(portfolio, risk_free_rate):
    avg_return = portfolio['returns'].mean()
    downside_returns = portfolio['returns'].copy()
    downside_returns[downside_returns > 0] = 0
    downside_std_dev = downside_returns.std()
    sortino = (avg_return - risk_free_rate) / downside_std_dev
    return sortino

In [ ]:
def max_drawdown(portfolio):
  window = len(portfolio)
  rolling_max = portfolio['total'].rolling(window, min_periods=1).max()
  daily_drawdown = portfolio['total']/rolling_max - 1.0
  max_daily_drawdown = daily_drawdown.rolling(window, min_periods=1).min()
  return max_daily_drawdown.min()

In [ ]:
def calmar_ratio(portfolio):
  avg_return = np.mean(portfolio['returns'])
  max_draw = max_drawdown(portfolio)
  calmar = avg_return / abs(max_draw)
  return calmar

In [ ]:
def bollinger_bands(portfolio, num_candles):
  window = 20
  trad
